In [2]:
from __future__ import division

import pandas as pd
import itertools
import numpy as np
import scipy.stats as sps
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')


ALPHA = 0.05

# Creating the data to be used for the experiement
group_1 = np.array([60.8, 57, 65, 58.6, 61.7])
group_2 = np.array([68.7, 67.7, 74, 66.3, 69.8])
group_3 = np.array([102.6, 102.1, 100.2, 96.5, np.nan])
group_4 = np.array([87.9, 84.2, 83.1, 85.7, 90.3])
df = pd.DataFrame({
        'group_1': group_1,
        'group_2': group_2,
        'group_3': group_3,
        'group_4': group_4
    })

#result_df = dataframe for the top table of the worksheet.
result_df = pd.DataFrame()

# Various things we need to calculate
result_df['count'] = df.count(axis=0)
result_df['sums'] = df.sum(axis=0)
result_df['means'] = df.mean(axis=0)
result_df['sum_x_squared'] = (df ** 2).sum(axis=0)
result_df['correction_term'] = (result_df['sums'] ** 2) / result_df['count']

# it's easier to add columns than rows, but now that it's all created, transpose the dataframe.
result_df = result_df.T
# creates the totals column.
result_df['sum_across_row'] = result_df.sum(axis=1)
# the means cell is not a sum, requires special handing.
result_df.loc['means', 'sum_across_row'] = result_df.loc['sums', 'sum_across_row'] / result_df.loc['count', 'sum_across_row']

print "PANEL A"
print result_df
print '-----------------------------------------------------'
print

# these create the various variables as defined on the worksheet
q0 = result_df.loc['count', 'sum_across_row']
q1 = result_df.loc['sums', 'sum_across_row']
avg_t = result_df.loc['means', 'sum_across_row']
q2 = result_df.loc['sum_x_squared', 'sum_across_row']
q3 = result_df.loc['correction_term', 'sum_across_row']

# These values are rounded to improving printing. May have a slight affect on downstream calculations, though...
q4 = np.round(q1 **2 / q0, 2)
q5 = np.round(q2 - q4, 2)
q6 = np.round(q3 - q4, 2)
q7 = np.round(q2 - q3, 2)

print "PANEL B"
print "CT: {}\tSSb: {}\t".format(q4, q6)
print "SSt: {}\tSSw: {}\t".format(q5, q7)
print '-----------------------------------------------------'
print

print "PANEL C"
# does the tests indicated in the third section.
if np.isclose(q5, q7 + q6):
    print "{} = {} + {}".format(q5, q7, q6)
else:
    print "q5 != q7 + q6"
if q4 <= q3 <= q2:
    print "{} <= {} <= {}".format(q4, q3, q2)
else:
    print "q4 ! <= q3 !<= q2"
if q6 <= q5:
    print "{} <= {}".format(q6, q5)
else:
    print "q6 ! <= q5"
    
print '-----------------------------------------------------'
print

# creating the reaminder of the variables in the output table
rows, groups = df.shape
degress_of_freedom_between = groups - 1
degrees_of_freedom_within = q0 - groups
degrees_of_freedom_total = q0 - 1
q8 = np.round(q6 / degress_of_freedom_between, 2)
q9 = np.round(q7 / degrees_of_freedom_within, 2)

# finally does the f test.
f_calc = np.round(q8 / q9, 2)

# I'm using the scipy.stats.f for calcuating the F crit.
f_crit = np.round(sps.f.isf(ALPHA, degress_of_freedom_between, degrees_of_freedom_within, loc=0, scale=1), 2)
# same here- using scipy.stats to calculate the p value. Due to some rounding, it may be slightly different
# from the excel calcuated p value.
p = sps.f.sf(f_calc, degress_of_freedom_between, degrees_of_freedom_within, loc=0, scale=1)

# This prints out the final panel. "\t" = tab.
print "PANEL D"
print "ANOVA TABLE OUTPUT"
print "SOURCE\tdf\tSS\t\tMS\t\tFcalc\t\tFcrit\t\tp-value"
print "BETWEEN\t{}\t{}\t\t{}\t\t{}\t\t{}\t\t{}".format(degress_of_freedom_between, q6, q8, f_calc, f_crit, p)
print "WITHIN\t{}\t{}\t\t{}".format(degrees_of_freedom_within, q7, q9)
print "TOTAL\t{}\t{}".format(degrees_of_freedom_total, q5)
print '-----------------------------------------------------'

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


PANEL A
                   group_1   group_2   group_3    group_4  sum_across_row
count                5.000      5.00      4.00      5.000       19.000000
sums               303.100    346.50    401.40    431.200     1482.200000
means               60.620     69.30    100.35     86.240       78.010526
sum_x_squared    18411.490  24046.71  40303.46  37220.240   119981.900000
correction_term  18373.922  24012.45  40280.49  37186.688   119853.550000
-----------------------------------------------------

PANEL B
CT: 115627.2	SSb: 4226.35	
SSt: 4354.7	SSw: 128.35	
-----------------------------------------------------

PANEL C
4354.7 = 128.35 + 4226.35
115627.2 <= 119853.55 <= 119981.9
4226.35 <= 4354.7
-----------------------------------------------------

PANEL D
ANOVA TABLE OUTPUT
SOURCE	df	SS		MS		Fcalc		Fcrit		p-value
BETWEEN	3	4226.35		1408.78		164.58		3.29		1.06419744941e-11
WITHIN	15.0	128.35		8.56
TOTAL	18.0	4354.7
-----------------------------------------------------


In [3]:
# Extra stuff for the bottom table. Performing multi-comparison pairwaise t tests.
for i, j in itertools.combinations(df.columns.values, 2):
    print "Comparing: {} to {}".format(i, j)
    t, p = sps.ttest_ind(df[df[i].notnull()][i].values, df[df[j].notnull()][j].values)
    print t, p
    print

Comparing: group_1 to group_2
-4.58023579012 0.00180145247564

Comparing: group_1 to group_3
-20.139435517 1.86346155016e-07

Comparing: group_1 to group_4
-13.5862076891 8.28084563001e-07

Comparing: group_2 to group_3
-16.1879738692 8.35217732889e-07

Comparing: group_2 to group_4
-9.19973096583 1.5765194269e-05

Comparing: group_3 to group_4
7.40220343263 0.000149143118802

